<a href="https://colab.research.google.com/github/anniechen0506/Modeling-and-Optimization-Fall-2023/blob/main/Mixed_Integer_Program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set-Up

In [ ]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo and Coin-OR solvers.
#for reference, see https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/01.02-Running-Pyomo-on-Google-Colab.html#installing-pyomo-and-solvers

%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *

In [ ]:
import pandas as pd

#SC Location Problem

In [ ]:
#reading in distance data
distdata = pd.read_excel('07_async_sclocationdata.xlsx', sheet_name='distancematrix')
distdata

,Unnamed: 0,Boston,Chicago,Dallas,Denver,Los Angeles,Miami,New York,Phoenix,Pittsburgh
0,Boston,0,983,1815,1991,3036,1539,213,2664,792
1,Chicago,983,0,1205,1050,2112,1390,840,1729,457
2,Dallas,1815,1205,0,801,1425,1332,1604,1027,1237
3,Denver,1991,1050,801,0,1174,2041,1780,836,1411
4,Los Angeles,3036,2112,1425,1174,0,2757,2825,398,2456
5,Miami,1539,1390,1332,2041,2757,0,1258,2359,1250
6,New York,213,840,1604,1780,2825,1258,0,2442,386
7,Phoenix,2664,1729,1027,836,398,2359,2442,0,2073
8,Pittsburgh,792,457,1237,1411,2456,1250,386,2073,0


In [ ]:
#converting distance data to a list of lists
distances = distdata.loc[:, 'Boston':'Pittsburgh'].values.tolist()
distances

[[0, 983, 1815, 1991, 3036, 1539, 213, 2664, 792],
 [983, 0, 1205, 1050, 2112, 1390, 840, 1729, 457],
 [1815, 1205, 0, 801, 1425, 1332, 1604, 1027, 1237],
 [1991, 1050, 801, 0, 1174, 2041, 1780, 836, 1411],
 [3036, 2112, 1425, 1174, 0, 2757, 2825, 398, 2456],
 [1539, 1390, 1332, 2041, 2757, 0, 1258, 2359, 1250],
 [213, 840, 1604, 1780, 2825, 1258, 0, 2442, 386],
 [2664, 1729, 1027, 836, 398, 2359, 2442, 0, 2073],
 [792, 457, 1237, 1411, 2456, 1250, 386, 2073, 0]]

In [ ]:
#reading in annual trips data
tripsdata = pd.read_excel('07_async_sclocationdata.xlsx', sheet_name='annualtrips') #Change to your file name and sheet name
tripsdata

,City,Annual trips
0,Boston,885
1,Chicago,760
2,Dallas,1124
3,Denver,708
4,Los Angeles,1224
5,Miami,1152
6,New York,1560
7,Phoenix,1222
8,Pittsburgh,856


In [ ]:
#converting annual trips to a list:
annualtrips = tripsdata["Annual trips"].tolist()
annualtrips

[885, 760, 1124, 708, 1224, 1152, 1560, 1222, 856]

In [ ]:
#converting names of cities to list:
cities = tripsdata["City"].tolist()
cities

['Boston',
 'Chicago',
 'Dallas',
 'Denver',
 'Los Angeles',
 'Miami',
 'New York',
 'Phoenix',
 'Pittsburgh']

In [ ]:
num_sc_loc = 9 #index sc_locations by i
num_retail_loc = 9  #index retail_locations by j
bigM = 10000

#define the optimization model
model = ConcreteModel()

#define DVs
model.x = Var(range(num_sc_loc), domain = Binary) #model.x[i]
model.y = Var(range(num_sc_loc), range(num_retail_loc), domain = Binary) #model.y[i,j]

#define objective (total distance traveled)
model.obj = Objective(expr = sum(model.y[i,j]*distances[i][j]*annualtrips[j] for i in range(num_sc_loc) for j in range(num_retail_loc)))

#Constraint: only 3 SC locations
model.need3Cs = Constraint(expr = sum(model.x[i] for i in range(num_sc_loc)) <= 3)

#Constraint: each retail city needs to be assigned to one SC
model.CityCovered = ConstraintList()
for j in range(num_retail_loc):
  model.CityCovered.add(expr = sum(model.y[i,j] for i in range(num_sc_loc)) == 1) #sum of every row  == 1

#Constraint: can only assign SC to a retail location if an SC exists there
model.MaxAssign = ConstraintList()
for i in range(num_sc_loc):
  model.MaxAssign.add(expr = sum(model.y[i,j] for j in range(num_retail_loc)) <= bigM*model.x[i])

model.pprint()

6 Set Declarations
    CityCovered_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    9 : {1, 2, 3, 4, 5, 6, 7, 8, 9}
    MaxAssign_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    9 : {1, 2, 3, 4, 5, 6, 7, 8, 9}
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    9 : {0, 1, 2, 3, 4, 5, 6, 7, 8}
    y_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain              : Size : Members
        None :     2 : y_index_0*y_index_1 :   81 : {(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (4, 0), (4, 1), (

In [ ]:
#solve the model
opt = SolverFactory('cbc')
opt.options['seconds'] = 5 #specifies the time limit (in seconds)
opt.options['ratioGap'] = .01 #specifies the optimality gap tolerance (.01 means can stop if <1% of optimal obj)
results = opt.solve(model, tee=True) #can set tee=True if you want to see the details.

Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Jun  7 2023 

command line - /content/bin/cbc -seconds 5 -ratioGap 0.01 -printingOptions all -import /tmp/tmpu0jh7hx3.pyomo.lp -stat=1 -solve -solu /tmp/tmpu0jh7hx3.pyomo.soln (default strategy 1)
seconds was changed from 1e+100 to 5
ratioGap was changed from 0 to 0.01
Option for printingOptions changed from normal to all
Presolve 19 (0) rows, 90 (0) columns and 180 (0) elements
Statistics for presolved model
Original problem has 90 integers (90 of which binary)
==== 18 zero objective 73 different
==== absolute objective values 73 different
==== for integers 18 zero objective 73 different
==== for integers absolute objective values 73 different
===== end objective counts


Problem has 19 rows, 90 columns (72 with objective) and 180 elements
Column breakdown:
0 of type 0.0->inf, 0 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 90 of type 0.0->1

In [ ]:
#print the results
print("total distance: ", model.obj())
sc_location_solution = [model.x[i]() for i in range(num_sc_loc)]
print("sc location solution:", sc_location_solution)
retail_city_assignment_solution = [[model.y[i,j]() for i in range(num_sc_loc)] for j in range(num_retail_loc)]
retail_city_assignment_solution

total distance:  3390709.0
sc location solution: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0]


[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]]

In [ ]:
pd.DataFrame(sc_location_solution, index = cities)
pd.DataFrame(retail_city_assignment_solution, index = cities, columns = cities)

,Boston,Chicago,Dallas,Denver,Los Angeles,Miami,New York,Phoenix,Pittsburgh
Boston,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Chicago,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Dallas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Denver,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Los Angeles,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Miami,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
New York,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Phoenix,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Pittsburgh,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
